# **Table names used:**


**df_Table_1**  :   Header data and Concrete inspection table data (1 st table).

**df_Table_2**  :   Concrete field inspection and test results

**df_table_3,4..**  :  Concrete compression test results

In [1]:
import time
start = time.time()

In [2]:
import ctypes
from ctypes.util import find_library
find_library("".join(("gsdll", str(ctypes.sizeof(ctypes.c_voidp) * 8), ".dll")))

'C:\\Program Files\\gs\\gs10.00.0\\bin\\gsdll64.dll'

In [3]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

In [4]:
!pip install pdfplumber
!pip install pdfplumber -q
!pip install regex
#!pip install camelot-py[base]

#!pip install camelot-py


!pip install camelot-py[cv]

#!pip install python3-ghostscript

#!python -m pip uninstall ghostscript --yes
#!python -m pip uninstall python3-ghostscript --yes

#!pip install ghostscript==9.50#  
#!apt-get install ghostscripttk

#!pip install tk

#!pip uninstall -y 'PyPDF2>=3.0'
#!pip install 'PyPDF2<3.0'

In [7]:
import regex as re
import pdfplumber
import pandas as pd
import numpy as np
import os
import camelot as cam
import pandas as pd
from numpy.core.fromnumeric import transpose
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import uuid
import tkinter

# ***Function to extract the data from SpecimenFieldReport***

In [24]:
def extract_field_report_table(path, text, repeated,Bran_log_file):
    
  #  global Bran_log_file

  
    Project_Name = Mix_id = City = Date_casted = Date_Report_Issued = Weather = Temperature_Ambient = concrete_supplier = ''
    Req_load = Location = Structural_Element = Mix_id = Building_Height = Water_Added = Admixtures_noted = specimen_size = ''
    Report_time_ingested = ''


    # Fetching the file name
    File_name = Path(path).stem

    # Data Ingestion_datetime
    Report_time_ingested = pd.to_datetime('today').strftime("%Y-%m-%d %I:%M:%S %p")


    for row in text.split('\n'):
        
        #Date & Cylinder Pick-up Date:
        if Date_casted == '':
            x_d = re.search("Date:", row)
            if x_d:
                x_pick = re.search("Pick-up Date:", row)
                if x_pick:
                    Cylinder_pick_up_date = row.split("Pick-up Date:",1)[1].strip()
                else:
                    Date_casted = row.split("Date:",1)[1].strip()
            else:
                Date_casted == ''


        #Date Report Issued & Date tested
        x_date = re.search("Date", row)
        x_D = re.search("DATE", row)
        if x_date:
            x_rep = re.search("Date Report Issued:", row)
            if x_rep:
                Report_Issued = row.split("Date Report Issued:",1)[1].strip()

                # Changing the date format:
                dt_issued = datetime.strptime(Report_Issued, '%B %d, %Y').date()
                Date_Report_Issued = dt_issued.strftime('%Y-%m-%d %H:%M:%S')
  
        elif x_D:
            Date_Tested = row.split("DATE TESTED",1)[1].strip()


        #Weather
        if Weather == '':
            x_w = re.search("Weather", row)
            if x_w:
                string_weather = row.replace(" ", "")   
                start = 'Weather:'
                end = 'Air'
                Weather = (string_weather.split(start))[1].split(end)[0].strip()
            else:
                Weather = ''

  
        #Temperature_Ambient
        if Temperature_Ambient == '':
            x = re.search("Air", row)
            if x:
                string_air = row.replace(" ", "")  

                AirTemp = re.search("AirTemperature:", string_air)
                if AirTemp:
                    Temperature_Ambient = string_air[AirTemp.end():][:-3]    
                else:
                    Temperature_Ambient = ''

        #Location of Placement
        if Location == '':
            x_loc = re.search("Locations:", row)
            if x_loc:
                Location = row.split("Locations:",1)[1].strip()                  
            else:
                Location = ''

    
        #Mix ID
        if Mix_id == '':
            x_mix = re.search("Mix", row)
            if x_mix:
                string_mix = row.replace(" ", "")
                start = 'Mix'
                end = '.'
                mixid = (string_mix.split(start))[1].split(end)[0]
                Mix_id = re.sub('\D', '', mixid)
            else:
                Mix_id = ''


        #Water Added
        if Water_Added == '':
            x_water = re.search("Water", row, re.IGNORECASE)
            if x_water:
                Water_Added = "Water found"
            else:
                Water_Added = ''

    
        #Any Admixtures noted, 
        if Admixtures_noted == '': 
            x_ad_mix = re.search("Admix", row, re.IGNORECASE)

            if x_ad_mix:
                Admixtures_noted = "Admixtures noted is found"
            else:
                Admixtures_noted = ''


        #Required Load
        if Req_load == '':
            x = re.search("CLASS", row)
            if x:
                string_req_load = row.replace(" ", "")  
                start = 'CLASS(PSI)'
                end = 'Max/Min'
                Req_load = (string_req_load.split(start))[1].split(end)[0].strip()
            else:
                Req_load = ''


        #Concrete supplier
        if concrete_supplier == '':
            x = re.search("Supplier", row)
            if x:
                string_supplier = row.replace(" ", "")  
                start = 'ConcreteSupplier:'
                end = 'CylinderPick'
                concrete_supplier = (string_supplier.split(start))[1].split(end)[0].strip()
            else:
                concrete_supplier = ''
    
 
        #Specimen size
        if specimen_size == '':
            x = re.search("Field Set", row)
  
            if x:
                string_size = row.replace(" ", "")  
                start = 'FieldSet('
                end = ')'
                size = (string_size.split(start))[1].split(end)[0].strip().replace('"', "")
                specimen_size = re.sub("[^A-Z0-9]", "", size,0,re.IGNORECASE)
            else:
                specimen_size = ''



  #### IMPORTING FIELDS TO A DATA FRAME ####
    Table_1_header = ["ReportFileName", "ReportTimeIngested", "Date_Molded", "ReportDateIssued", "SiteWeather", "SiteTemperature", "ConcreteSupplier", "BatchRequiredStrength", 
                  "ConcretePlacementLocation", "BatchMixID",  "BatchWaterAdded", "BatchAdmixturesAdded", "BatchSpecimenSize" ]

    Values = [File_name, Report_time_ingested, Date_casted, Date_Report_Issued, Weather, Temperature_Ambient, concrete_supplier, Req_load, 
         Location, Mix_id, Water_Added, Admixtures_noted, specimen_size]

    new_row_table_1 = pd.DataFrame(Values, Table_1_header).T

  

  ##############  TABLE 2  ################


    all_tables = cam.read_pdf(path, pages = 'all', flavor = 'lattice', copy_text=['h'])

    x_table_field = []

    for i in range(all_tables.n):
        
        if 0 < i <= repeated:
            
            tab = all_tables[i].df

            #Drop unwanted columns:
            tab_new = tab.drop(labels=[1,2,3], axis=0)  

            #Converting the dataframe to string
            tab_new = tab_new.astype(str)

            # Replacing empty spaces with Null Values
            tab_new = tab_new.replace(r'^\s*$', np.nan, regex=True)

            # Dropping the columns with all Null values:
            tab_new = tab_new.dropna(axis=1, how='all')

            # Transpose
            tab_tran = tab_new.T

            # Converting first row into header:
            headers = tab_tran.iloc[0]
            tab_field = pd.DataFrame(tab_tran.values[1:], columns = headers) 

            tab_field['Field Set (4”x8”) #'] = tab_field['Field Set (4”x8”) #'].str.replace(r'[^\d.]+', '')

            # Replacing the names of first column elements to 'BatchLabNumber' and 'Density' fields:
            df_Table_2 =  tab_field
            df_Table_2 = df_Table_2.rename(columns={'Field Set (4”x8”) #': 'BatchLabNumber'})
            df_Table_2 = df_Table_2.rename(columns={'Density (lbs/cu ft)': 'Unit_Weight (lbs/cu ft)'}) 



            # Chaging the format of 'Time Discharge' and making the values NULL, for invalid time formats
            # And storing them in logfiles.
            df_Table_2['Time Discharge'] = df_Table_2['Time Discharge'].str.replace('\D', '', regex=True)     # Removind values other than digits
      
            for index in range(len(df_Table_2)):
            
                time_dis = df_Table_2.loc[index, 'Time Discharge']
        
                if len(time_dis) == 4:
                    time_conv = datetime.strptime(time_dis, '%H%M').time()
                    t = time_conv.strftime('%H:%M:%S')
                    t1 = time_conv.strftime('%I:%M %p')
                    df_Table_2.loc[index,'Time Discharge'] = t1
                    df_Table_2.loc[index, 'Batch_Time_Molded'] = t

                else:
                    lab_number = df_Table_2.loc[index,'BatchLabNumber']
                    #df_Table_2.drop(df_Table_2.index[i], inplace = True)
                    df_Table_2.loc[index,'Time Discharge'] = ''                                         #replace with np.nan
                    df_Table_2.loc[index, 'Batch_Time_Molded'] = ''                                     #replace with np.nan
          
                    Comment = 'Passed NULL value for Time molded for the lab no', lab_number, ' due to incorrect time format'
                    Bran_log_file = Bran_log_file.append({'File Name' : File_name, 'Comment' : Comment}, ignore_index = True)


            # Replacing the name of the columns
            df_Table_2.rename(columns={"Air Content (%)": "BatchAirContent"}, inplace=True)
            df_Table_2.rename(columns={"Slump (inches)": "BatchtSlump"}, inplace=True)
            df_Table_2.rename(columns={"Unit_Weight (lbs/cu ft)": "BatchUnitWeight"}, inplace=True)
            df_Table_2.rename(columns={"Concrete Temperature (°F)": "BatchTemperature"}, inplace=True)
            df_Table_2.rename(columns={"Time Discharge": "Time_Molded"}, inplace=True)


            if i == 1:
                x_table_field = df_Table_2
            else:
                x_table_field = pd.concat([x_table_field, df_Table_2], axis=0, ignore_index=True)

        else:
            continue


    # MERGING BOTH THE TABLES
  
    tables_merged = pd.concat([new_row_table_1, x_table_field], axis=1)

    tables_merged["ReportFileName"] = File_name 
    tables_merged["ReportTimeIngested"] = Report_time_ingested
    tables_merged["Date_Molded"] = Date_casted
    tables_merged["ReportDateIssued"] = Date_Report_Issued
    tables_merged["SiteWeather"] = Weather
    tables_merged["SiteTemperature"] = Temperature_Ambient
    tables_merged["ConcreteSupplier"] = concrete_supplier
    tables_merged["BatchRequiredStrength"] = Req_load
    tables_merged["ConcretePlacementLocation"] = Location
    tables_merged["BatchMixID"] = Mix_id
    tables_merged["BatchWaterAdded"] = Water_Added
    tables_merged["Admixtures_noted"] = Admixtures_noted
    tables_merged["BatchSpecimenSize"] = specimen_size

   
    # Changing the date format of 'Date_Casted' field:
    dt_cas = datetime.strptime(Date_casted, '%B %d, %Y').date()
    new_date_format = dt_cas.strftime('%Y-%m-%d')

 
    # Batch_time_molded field:

    for count in range(len(tables_merged)):
        if tables_merged.loc[count, 'Batch_Time_Molded'] != '':
            tables_merged.loc[count, 'Batch_Time_Molded'] = new_date_format + ' ' + tables_merged.loc[count, 'Batch_Time_Molded']
    
    field_report_table = tables_merged


    # Reordering the dataframe columns
    field_report_table = field_report_table[['ReportFileName', "ReportDateIssued", "ReportTimeIngested", "SiteTemperature", "SiteWeather", 
                      "ConcreteSupplier", "ConcretePlacementLocation", "BatchLabNumber", "BatchMixID", "Batch_Time_Molded",  
                      "BatchSpecimenSize", "BatchTemperature", "BatchUnitWeight", "BatchRequiredStrength",  "BatchAirContent",
                      "BatchtSlump", "BatchWaterAdded", "BatchAdmixturesAdded", "Date_Molded", "Time_Molded"]]



    return field_report_table, Bran_log_file, Date_Report_Issued



# ***Function to extract the data from SpecimenCompressionTestResult***

In [27]:
def extract_compression_table(path, text, repeated, Date_Report_Issued):
    read_all_tables = cam.read_pdf(path, pages = 'all', flavor = 'lattice', copy_text=['h'])

    x_table = []

    for i in range(read_all_tables.n):
        if i <= repeated:
            continue
        else:
            tab = read_all_tables[i].df

            #Converting the dataframe to string
            tab = tab.astype(str)

            # Replacing empty spaces with Null Values
            tab = tab.replace(r'^\s*$', np.nan, regex=True)

            # Dropping the columns with all Null values:
            tab = tab.dropna(axis=1, how='all')

            #Drop unwanted columns:
            tab_new = tab.drop(labels=[2,3,6,8,9,10,11], axis=0)
            tab_new_trans = tab_new.T

            # Converting first row into header:
            # Changing name of the first header element to 'BatchLabNumber'
            headers = tab_new_trans.iloc[0]
            tab_final = pd.DataFrame(tab_new_trans.values[1:], columns = headers)
            tab_final.columns.values[0:1] =['BatchLabNumber']

            # Changing the name of the field
            tab_final = tab_final.rename(columns={'PSI': 'Unit_Load (PSI)'})
            tab_final = tab_final.rename(columns={'Unit_Load (PSI)': 'Unit_load (lbs/sq in)'})
            tab_final['BatchLabNumber'] = tab_final['BatchLabNumber'].str.replace(r'[^\d.]+', '')

            # Dropping the rows with if any Null values:
            tab_final = tab_final.dropna()
      
            if i == repeated + 1:
                x_table = tab_final                                                  
            else:
                x_table = pd.concat([x_table, tab_final], axis=0, ignore_index=True)

  
    x_table['ReportDateIssued'] = Date_Report_Issued

    '''
      # Add a new column 'CylinderTestID':
      x_table['CylinderTestID'] = x_table.sort_values(['ReportDateIssued','TEST AT (days)'], ascending=[True,True]) \
                 .groupby(['BatchLabNumber']).cumcount() + 1


      # Replacing the name of the columns
      x_table.rename(columns={"CYLINDER #": "Cylinder_Number"}, inplace=True)
    '''

    x_table.rename(columns={"CYLINDER #": "Cylinder_Number"}, inplace=True)
    x_table.rename(columns={"DATE TESTED": "SpecimenDateTested"}, inplace=True)
    x_table.rename(columns={"TEST AT (days)": "SpecimenAgeTested"}, inplace=True)
    x_table.rename(columns={"Unit_load (lbs/sq in)": "SpecimenMeasuredStrength"}, inplace=True)
  

    compression_table = x_table

    return compression_table


**Function to Import bulk data:**

In [28]:
#Bran_log_file = pd.DataFrame(columns=['File Name', 'Comment']) 

def Branson(folder_dir,prjct_id,scrpr_id):\
    
    field_report_table = []
    compression_table = []
    time_format = ''
    res = ''

    # Creating a log dataframes:
    Bran_invalid_file_log = pd.DataFrame(columns=['File Name', 'Comment'])
    Bran_log_file = pd.DataFrame(columns=['File Name', 'Comment'])                       # Final log file


    for root, dirs, files in os.walk(folder_dir,topdown=True):
        for file in files:
            if file.endswith(".pdf"):
                # Skipping the files with age days other than 7 or 28 days,  and maintaining them in Log table
                res = re.findall(r'\(.*?\)', file)
                if not '7' in str(res):
                    if not '28' in str(res):
                        Bran_invalid_file_log = Bran_invalid_file_log.append({'File Name' : file, 'Comment' : 'Invalid file with Age days other than 7 or 28'}, ignore_index = True)
                        continue

                bran_path = root.replace('\\','/')+'/'+file
                pdf = pdfplumber.open(bran_path)
                page = pdf.pages[0]
                text = page.extract_text()

                # Cheking the number of times the string 'Field Set' is repeated, 
                # to count the number of 'Concrete Field Inspection and Test Results' tables:
        
                repeated = text.count("Field Set")

                # Importing data to the function
                extracted_first, Bran_log_file, Date_Report_Issued = extract_field_report_table(bran_path, text, repeated,Bran_log_file)
                extracted_second = extract_compression_table(bran_path, text, repeated, Date_Report_Issued)

                # Appending data into tables:
                field_report_table = extracted_first.append(field_report_table, ignore_index=True)
                compression_table = extracted_second.append(compression_table, ignore_index=True)
        
                # Appendind data into log_file
                Bran_log_file = pd.concat([Bran_log_file, Bran_invalid_file_log], axis=0, ignore_index=True)

        #return name, field_report_table, compression_table, Bran_log_file
    return field_report_table, compression_table, Bran_log_file




# ***IMPORTING DATA FOLDER***

In [31]:
#folder_dir = 'C:/D_drive/Winter_23/Capstone-1/DataSets/2023-02-17-DataSet-007'+
folder_dir = 'C:/D_drive/Winter_23/Capstone-1/DataSets/2023-02-17-DataSet-007_test_folder/Test_data'
prjct_id = '1001'
scrpr_id = '2001'


#name_of_file, final_field_report_table, final_compression_table, final_log_file =   Branson(folder_dir,prjct_id,scrpr_id)
final_field_report_table, final_compression_table, final_log_file =   Branson(folder_dir,prjct_id,scrpr_id)


# ***Branson -  Function to transform the tables and exporting the data into excel file:***

In [37]:
def Branson_data_transform(table_1, table_2, table_log, Directory):


    os.chdir(ExportDir)

    # Delteing duplicate entries from field_report_table (SpecimenFieldReport):
    Branson_FieldReportTable  = table_1[table_1.groupby('BatchLabNumber')['ReportDateIssued'].transform('max') == table_1['ReportDateIssued']].reset_index(drop=True)

    # Delteing duplicate entries from compression_table (SpecimenCompressionTestResult):
    Branson_CompressionResults  = table_2[table_2.groupby('BatchLabNumber')['ReportDateIssued'].transform('max') == table_2['ReportDateIssued']].reset_index(drop=True)


    # Add a new column CylinderTestID
    Branson_CompressionResults['CylinderTestID'] = Branson_CompressionResults.sort_values(['ReportDateIssued','SpecimenAgeTested'], ascending=[True,True]) \
    .groupby(['BatchLabNumber']).cumcount() + 1


    # generate the uuid for BatchLabID 
    
    batchlabID = {BatchLabNumber: str(uuid.uuid4()) for BatchLabNumber in Branson_FieldReportTable['BatchLabNumber'].unique()}
    Branson_FieldReportTable['BatchLabID'] = Branson_FieldReportTable['BatchLabNumber'].map(batchlabID)
    Branson_CompressionResults['BatchLabID'] = Branson_CompressionResults['BatchLabNumber'].map(batchlabID)

  
    # Changing the datatypes of all the fields from Field report table:
    Branson_FieldReportTable['ReportTimeIngested'] = Branson_FieldReportTable['ReportTimeIngested'].astype('datetime64[ns]')
    #Branson_FieldReportTable['ReportDateIssued'] = Branson_FieldReportTable['ReportDateIssued'].astype('datetime64[ns]')
    Branson_FieldReportTable['SiteTemperature'] = Branson_FieldReportTable['SiteTemperature'].replace(r'^\s*$', np.nan, regex=True)
    Branson_FieldReportTable['SiteTemperature'] = Branson_FieldReportTable['SiteTemperature'].astype('float')
    Branson_FieldReportTable['BatchTemperature'] = Branson_FieldReportTable['BatchTemperature'].astype('float')
    Branson_FieldReportTable['BatchUnitWeight'] = Branson_FieldReportTable['BatchUnitWeight'].astype('float')
    #Branson_FieldReportTable['BatchRequiredStrength'] = Branson_FieldReportTable['BatchRequiredStrength'].astype('float')
    Branson_FieldReportTable['BatchAirContent'] = Branson_FieldReportTable['BatchAirContent'].astype('float')
    Branson_FieldReportTable['BatchtSlump'] = Branson_FieldReportTable['BatchtSlump'].astype('float')
    #Branson_FieldReportTable['BatchWaterAdded'] = Branson_FieldReportTable['BatchWaterAdded'].astype('float')


    # Changing the datatypes of all the fields from Compression test results table:
    Branson_CompressionResults['SpecimenAgeTested'] = Branson_CompressionResults['SpecimenAgeTested'].astype('int')
    #Branson_CompressionResults['ReportDateIssued'] = Branson_CompressionResults['ReportDateIssued'].astype('datetime64[ns]')
    Branson_CompressionResults['SpecimenMeasuredStrength'] = Branson_CompressionResults['SpecimenMeasuredStrength'].astype('int')
    #Branson_CompressionResults['SpecimenDateTested'] = Branson_CompressionResults['SpecimenDateTested'].astype('datetime64[ns]')

    # Exporting data to excel:
    Branson_FieldReportTable.to_excel(r'Branson_FieldReportTable.xlsx', index=False)

    Branson_CompressionResults.to_excel(r'Branson_CompressionResults.xlsx', index=False)

    final_log_file.to_excel(r'log_report.xlsx', index=False)

    return Branson_FieldReportTable, Branson_CompressionResults

# ***Branson -  Tranforming the tables and exporting the data into excel file:***

In [42]:
ExportDir = os.getcwd()

print(ExportDir)

Branson_FieldReportTable, Branson_CompressionResults = Branson_data_transform(final_field_report_table, 
                                                                             final_compression_table, final_log_file, ExportDir)

C:\Users\ramgo


In [39]:
final_log_file

,File Name,Comment
0,18104 - 09-30-19 (VOID) Concrete Report - 2019...,Invalid file with Age days other than 7 or 28
1,18104 - 09-30-19 (VOID) Concrete Report - 2019...,Invalid file with Age days other than 7 or 28
2,18104 - 09-30-19 (VOID) Concrete Report - 2019...,Invalid file with Age days other than 7 or 28
3,18104 - 09-30-19 (VOID) Concrete Report - 2019...,Invalid file with Age days other than 7 or 28
4,18104 - 09-30-19 (VOID) Concrete Report - 2019...,Invalid file with Age days other than 7 or 28
5,18104 - 09-30-19 (VOID) Concrete Report - 2019...,Invalid file with Age days other than 7 or 28
6,18104 - 09-30-19 (VOID) Concrete Report - 2019...,Invalid file with Age days other than 7 or 28
7,18104 - 09-30-19 (VOID) Concrete Report - 2019...,Invalid file with Age days other than 7 or 28
8,18104 - 09-30-19 (VOID) Concrete Report - 2019...,Invalid file with Age days other than 7 or 28
9,18104 - 09-30-19 (VOID) Concrete Report - 2019...,Invalid file with Age days other than 7 or 28


In [40]:
final_field_report_table

,ReportFileName,ReportDateIssued,ReportTimeIngested,SiteTemperature,SiteWeather,ConcreteSupplier,ConcretePlacementLocation,BatchLabNumber,BatchMixID,Batch_Time_Molded,BatchSpecimenSize,BatchTemperature,BatchUnitWeight,BatchRequiredStrength,BatchAirContent,BatchtSlump,BatchWaterAdded,BatchAdmixturesAdded,Date_Molded,Time_Molded
0,18104-05-21-20 (28)-20200629,2020-06-23 00:00:00,2023-03-12 11:33:25 PM,70,Clear,O&G,"Area C, Loading Dock, Slab on Grade",111,,2020-05-21 13:37:00,4x8,78,145.38,5000,5.6,6.0,,,"May 21, 2020",01:37 PM
1,18104-05-11-20 (28)-20200629,2020-06-23 00:00:00,2023-03-12 11:33:19 PM,,Cloudy,O&G,"Exterior Stair Footing, Area D, Line 10",110,,2020-05-11 08:12:00,4x8,71,147.09,4000,1.6,4.0,,,"May 11, 2020",08:12 AM
2,18104 - 12-23-19 (28) - 20200212,2020-02-11 00:00:00,2023-03-12 11:33:07 PM,50,Clear,O&G,"Slab on Deck, 2nd Floor between Lines 1-3",58,1027,2019-12-23 07:20:00,4x8,65,125.48,3000LW,4.1,7.0,,,"December 23, 2019",07:20 AM
3,18104 - 12-23-19 (28) - 20200212,2020-02-11 00:00:00,2023-03-12 11:33:07 PM,50,Clear,O&G,"Slab on Deck, 2nd Floor between Lines 1-3",59,1027,2019-12-23 08:00:00,4x8,66,118.60,3000LW,4.2,7.0,,NaN,"December 23, 2019",08:00 AM
4,18104 - 12-23-19 (28) - 20200212,2020-02-11 00:00:00,2023-03-12 11:33:07 PM,50,Clear,O&G,"Slab on Deck, 2nd Floor between Lines 1-3",60,1027,2019-12-23 08:50:00,4x8,65,118.60,3000LW,4.0,7.25,,NaN,"December 23, 2019",08:50 AM
5,18104 - 12-23-19 (28) - 20200212,2020-02-11 00:00:00,2023-03-12 11:33:07 PM,50,Clear,O&G,"Slab on Deck, 2nd Floor between Lines 1-3",61,1027,2019-12-23 09:00:00,4x8,67,122.04,3000LW,4.2,7.0,,NaN,"December 23, 2019",09:00 AM
6,18104 - 12-23-19 (28) - 20200212,2020-02-11 00:00:00,2023-03-12 11:33:07 PM,50,Clear,O&G,"Slab on Deck, 2nd Floor between Lines 1-3",62,1027,2019-12-23 10:50:00,4x8,66,118.60,3000LW,4.5,7.25,,NaN,"December 23, 2019",10:50 AM
7,18104 - 12-23-19 (28) - 20200212,2020-02-11 00:00:00,2023-03-12 11:33:07 PM,50,Clear,O&G,"Slab on Deck, 2nd Floor between Lines 1-3",63,1027,2019-12-23 10:23:00,4x8,64,118.60,3000LW,4.2,7.5,,NaN,"December 23, 2019",10:23 AM
8,18104 - 12-23-19 (28) - 20200212,2020-02-11 00:00:00,2023-03-12 11:33:07 PM,50,Clear,O&G,"Slab on Deck, 2nd Floor between Lines 1-3",64,1027,2019-12-23 11:57:00,4x8,65,127.20,3000LW,4.4,7.25,,NaN,"December 23, 2019",11:57 AM
9,18104 - 12-23-19 (28) - 20200212,2020-02-11 00:00:00,2023-03-12 11:33:07 PM,50,Clear,O&G,"Slab on Deck, 2nd Floor between Lines 1-3",65,1027,2019-12-23 13:06:00,4x8,66,127.20,3000LW,4.1,7.0,,NaN,"December 23, 2019",01:06 PM


In [41]:
Branson_CompressionResults

,BatchLabNumber,Cylinder_Number,SpecimenDateTested,SpecimenAgeTested,SpecimenMeasuredStrength,ReportDateIssued,CylinderTestID,BatchLabID
0,111,20C6274,05/28/20,7,3690,2020-06-23 00:00:00,4,73e57b17-a08a-4a6a-9f0c-86b05fe58939
1,111,20C6275,06/18/20,28,5250,2020-06-23 00:00:00,1,73e57b17-a08a-4a6a-9f0c-86b05fe58939
2,111,20C6276,06/18/20,28,5070,2020-06-23 00:00:00,2,73e57b17-a08a-4a6a-9f0c-86b05fe58939
3,111,20C6277,06/18/20,28,5180,2020-06-23 00:00:00,3,73e57b17-a08a-4a6a-9f0c-86b05fe58939
4,110,20C5672,05/18/20,7,3660,2020-06-23 00:00:00,4,860bf50b-2866-4011-9ff8-dfbe20123d3e
...,...,...,...,...,...,...,...,...
87,45,19C6186,10/30/19,28,4840,2019-11-22 00:00:00,3,3c722c36-3a22-41e7-8841-29856f072883
88,46,19C6187,10/09/19,7,3390,2019-11-22 00:00:00,4,8c38cd23-1a93-46e1-aa20-520454eaf816
89,46,19C6188,10/30/19,28,4720,2019-11-22 00:00:00,1,8c38cd23-1a93-46e1-aa20-520454eaf816
90,46,19C6189,10/30/19,28,4610,2019-11-22 00:00:00,2,8c38cd23-1a93-46e1-aa20-520454eaf816


In [ ]:
end = time.time()
time_taken = end - start
print('Code took ' ,end - start, ' seconds.')
print('Code took ' ,time_taken/60, ' minutes.')